In [5]:
import pandas as pd
import json
import dotenv
import os
import openai
from openai import OpenAI, AsyncOpenAI
import asyncio
import numpy as np
from prompt import SYSTEM_PROMPT


dotenv.load_dotenv()

client = OpenAI(
  project=os.environ.get('PROJECT_ID'),
  api_key=os.environ.get('OPENAI_API_KEY')
)

In [7]:
# utils

def get_embedding(text: str, model="text-embedding-3-small", **kwargs):
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [8]:
for file in ['../cleaned_equal.csv', '../cleaned_non_equal.csv']:
    with open(file.replace('.csv', '.json'), 'w') as json_file:
        json.dump(pd.read_csv(file).set_index('id').T.to_dict(), json_file, indent=4)

In [35]:
unequal_dict = json.load(open('../cleaned_non_equal.json'))

_key = '9dbf2aa9-52d5-4952-aa42-a01e1fc344c8'

for key, value in unequal_dict[_key].items():
    print(key,":", value)

prev_context : Customer's Message: Hello, I am so glad you have reached out.  I am having a problem with one of the new hearing aides, no matter if it’s one ear or the other I cannot stop it from making sounds, I have tried different domes, different ears, etc.  I know think this one may be defective.

On another note, I purchased hearing aides from you in <DATE_TIME>, <DATE_TIME>, I lost one of them and can’t find it.  It is a different design from the new ones.  Can it be replaced?  I really liked them.  If fact because the new ones have only one that is working right, I am wear two different design ones at the moment.

Hope you can help.

<PERSON>

response : Hi <PERSON>,

Thank you for reaching out. I've created a replacement order for the defective hearing aid and sent a return label to your email. Please return it with all accessories. Regarding the lost hearing aid, we can discuss replacement options. Let me know if you need further assistance.

Best regards,
Brashna K.
agent_re

In [12]:
unequal_dict = json.load(open('../cleaned_non_equal.json'))
keys_list = list(unequal_dict.keys())

CHATS = []
for idx in range(len(keys_list)):
    part = unequal_dict[keys_list[idx]]
    COMPLETE_CHAT = r"""PREV_CONTEXT: {} RESPONSE1: {} RESPONSE2: {}""".format(part['prev_context'], part['response'], part['agent_response'])
    CHATS.append(COMPLETE_CHAT)

ai_response_embeds = [get_embedding(value['response']) for value in list(unequal_dict.values())[:10]]
agent_response_embeds = [get_embedding(value['agent_response']) for value in list(unequal_dict.values())[:10]]

diff = [{keys_list[idx]: cosine_similarity(ai_response_embeds[idx], agent_response_embeds[idx])} for idx in range(len(ai_response_embeds))]

In [13]:
diff

[{'9eb89242-c9a8-496f-aa5a-78aee3b1724e': 0.7130335959817298},
 {'515d6d85-8d05-4bbd-b8c8-2d81f7f9e1ad': 0.8456429100140704},
 {'2ebd2e1f-8497-4614-9370-a871434f5d14': 0.6300213842666598},
 {'ef45bcd2-0cc4-4d36-b895-a37913f3b602': 0.947483529183924},
 {'61f63f12-1854-4e64-8fcb-9b39968bd123': 0.9859149656653425},
 {'a3d9f113-4881-44c8-8f87-ab0f8c04a9f9': 0.9215406247040218},
 {'c8648fc7-0677-4db7-ab65-46bb94077599': 0.6615057795294927},
 {'35ec0581-9a07-46a9-b515-4255fc6a52ff': 0.7898228418783072},
 {'b42513d6-17af-45f1-a8b2-7138deacf2d8': 0.6878579046633595},
 {'0126184a-d81f-4b04-a026-f1cde1ac9423': 0.6580592996372631}]

In [5]:
# print(CHATS[0])
# CHATS[0]

In [6]:
# metrics = ['clarity_and_directness', 'completeness','relevance', 'tone_and_customer_friendliness']

# issues_with_ai_response = {
#     '2': 'very long and complex',
#     '6': 'incomplete information',
#     '7': 'incomplete information',
#     '8': 'unnessary information like company name at the end',
    
#     }

In [10]:
responses = []

cnt = 1
for chat in CHATS[20:30]:

    system_message = {"role": "system", "content": SYSTEM_PROMPT.replace(">>chat<<", chat)}

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[system_message],
        response_format={"type": "json_object"},
    )
    content = json.loads(response.choices[0].message.content)
    usage = response.usage.to_dict()
    responses.append(
        {   
            "chat": chat,
            "content": content,
            "usage": usage,
        }
    )
    print(cnt, "out of 5")
    cnt += 1


1 out of 5
2 out of 5
3 out of 5
4 out of 5
5 out of 5
6 out of 5
7 out of 5
8 out of 5
9 out of 5
10 out of 5


In [ ]:
for res in responses:
    # print(res['content'])
    if res["content"]["response1"]["overall"] < res["content"]["response2"]["overall"]:
        print('/////////////////////////////////////////////////////////', "\n\n\n")
        print(res['chat'], "\n\n\n")
        print(res['content'], "\n\n\n")

In [ ]:
examples_of_incomplete_information = ['eccbcd59-9f95-43da-a253-3c502f98b26a', 'c342dc8b-f30f-469a-9b44-e1090e61d48b']
examples_of_incomplete_customer_response = ['f8c8a244-daf9-46c8-be8c-9f6e8907e619']